In [1]:
%cd ..


C:\Users\benza-ord\source\repos\CS583-Research-Project


In [2]:
import pandas as pd
import numpy as np


import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 4)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)
        

1 Physical GPUs, 1 Logical GPUs


In [3]:
df = pd.read_csv("cxs583_cleaned_file_v2.csv")
df.head()

,date,time,tweet,class,result,tweet_length,tweet_cleaned,tweet_clst
0,2012-10-16 00:00:00,10:28:53-05:00,"Kirkpatrick, who wore a baseball cap embroider...",0,NaN,136,"['Kirkpatrick', 'wear', 'baseball', 'cap', 'em...",Kirkpatrick who wore a baseball cap embroidere...
1,2012-10-16 00:00:00,10:04:30-05:00,#<e>obama</e> debates that Cracker Ass Cracker...,1,NaN,88,"['obama', 'debate', 'Cracker', 'Ass', 'Cracker...",obama debates that Cracker Ass Cracker tonight...
2,2012-10-16 00:00:00,09:50:08-05:00,@Hollivan @hereistheanswer Youre missing the ...,0,NaN,140,"['Hollivan', 'hereistheanswer', 'Youre', 'miss...",Hollivan hereistheanswer Youre missing the poi...
3,2012-10-16 00:00:00,10:00:16-05:00,I was raised as a Democrat left the party yea...,-1,NaN,146,"['raise', 'Democrat', 'leave', 'party', 'years...",I was raised as a Democrat left the party year...
4,2012-10-16 00:00:00,09:48:07-05:00,The <e>Obama camp</e> can't afford to lower ex...,0,NaN,124,"['Obama', 'camp', 'ca', 'afford', 'lower', 'ex...",The Obama camp ca n't afford to lower expectat...


In [4]:
#Convert tweet_clst to string
df['tweet_clst'] = df['tweet_clst'].astype(str)

# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each tweet.
MAX_SEQUENCE_LENGTH = 75
# This is fixed.
EMBEDDING_DIM = 150

In [5]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
# split 90-10 train-test
df_train, df_test = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['tweet_clst'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 18759 unique tokens.


In [8]:
X = tokenizer.texts_to_sequences(df_train['tweet_clst'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (10145, 75)


In [9]:
Y = pd.get_dummies(df_train['class']).values
print('Shape of label tensor:', Y.shape)


Shape of label tensor: (10145, 3)


In [10]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Flatten, Dense, Embedding, LSTM, SpatialDropout1D, Conv1D, GlobalMaxPooling1D, Bidirectional,TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model

#model = Sequential()
#model.add(Embedding(MAX_VOCAB_SIZE, 200, input_length=X.shape[1]))
#model.add(Bidirectional(LSTM(200)))
#model.add(Dropout(0.2))
#model.add(Dense(3, activation='relu'))
#model.add(Dense(3, activation='sigmoid'))
#model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])


# main model
input = Input(shape=(MAX_SEQUENCE_LENGTH,))
model = Embedding(MAX_VOCAB_SIZE, 200, input_length=MAX_SEQUENCE_LENGTH)(input)
model =  Bidirectional (LSTM (100,return_sequences=True,dropout=0.50),merge_mode='concat')(model)
model = TimeDistributed(Dense(100,activation='relu'))(model)
model = Flatten()(model)
model = Dense(100,activation='relu')(model)
output = Dense(3,activation='softmax')(model)

model = Model(input,output)
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])


# run for small number of epochs then save 
epochs = 10

history = model.fit(X, Y, epochs=epochs)


Epoch 1/10
318/318 [==============================] - 21s 67ms/step - loss: 0.9827 - accuracy: 0.5171
Epoch 2/10
318/318 [==============================] - 21s 65ms/step - loss: 0.7021 - accuracy: 0.7018
Epoch 3/10
318/318 [==============================] - 21s 66ms/step - loss: 0.3932 - accuracy: 0.8462
Epoch 4/10
318/318 [==============================] - 21s 66ms/step - loss: 0.2158 - accuracy: 0.9215
Epoch 5/10
318/318 [==============================] - 21s 65ms/step - loss: 0.1332 - accuracy: 0.9527
Epoch 6/10
318/318 [==============================] - 21s 65ms/step - loss: 0.0871 - accuracy: 0.9690
Epoch 7/10
318/318 [==============================] - 22s 71ms/step - loss: 0.0831 - accuracy: 0.9689
Epoch 8/10
318/318 [==============================] - 21s 66ms/step - loss: 0.0597 - accuracy: 0.9764
Epoch 9/10
318/318 [==============================] - 21s 67ms/step - loss: 0.0487 - accuracy: 0.9827
Epoch 10/10
318/318 [==============================] - 21s 65ms/step - loss: 0.043

In [23]:
X_test = tokenizer.texts_to_sequences(df_test['tweet_clst'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['class']).values
print('Shape of label tensor:', Y_Test.shape)

loss, accuracy = model.evaluate(X_test, Y_Test)

print("loss",loss)
print("accuracy",accuracy)



Shape of data tensor: (1128, 75)
Shape of label tensor: (1128, 3)
36/36 [==============================] - 0s 5ms/step - loss: 3.1580 - accuracy: 0.5328
loss 3.1579997539520264
accuracy 0.5328013896942139


In [25]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = model.predict(X_test)

# get the class with highest probability for each sample
y_pred = np.argmax(predictions, axis=-1) - 1

print(classification_report(df_test['class'], y_pred))

              precision    recall  f1-score   support

          -1       0.58      0.60      0.59       455
           0       0.48      0.54      0.51       380
           1       0.54      0.41      0.46       293

    accuracy                           0.53      1128
   macro avg       0.53      0.52      0.52      1128
weighted avg       0.53      0.53      0.53      1128



In [26]:
y_pred

array([1, 0, 0, ..., 1, 1, 1], dtype=int64)

In [39]:
    
outF = open("obama.txt", "w")
STUDENT_ID= 15
# PRINT STUDENT ID 
outF.write(str(STUDENT_ID))
outF.write("\n")

for idx, pred in zip(range(len(y_pred)), y_pred):
  # write line to output file
    outF.write(str(idx)+";;"+str(pred))
    outF.write("\n")
outF.close()